In [1]:
import csv
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
def save_data(lips, Y, data_type):
    with open(f'{data_type}_lips.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        header = ['label']
        for i in range(lips[0].shape[0]): #frames
            for j in range(lips[0].shape[1]): #height
                for k in range(lips[0].shape[2]): #width
                    header.append(f'pixel_{i}_{j}_{k}')
        writer.writerow(header)
        for i in range(len(lips)):
            row = [Y[i].argmax()]
            current_lips = lips[i].flatten()
            for j in range(len(current_lips)):
                row.append(current_lips[j])
            writer.writerow(row)
            
            
def load_data():
    X_train = pd.read_csv('train_lips.csv')
    Y_train = X_train['label']
    del X_train['label']
    X_valid = pd.read_csv('valid_lips.csv')
    Y_valid = X_valid['label']
    del X_valid['label']
    
    X_test = pd.read_csv('test_lips.csv')
    Y_test = X_test['label']
    del X_test['label']
    return X_train, Y_train, X_valid, Y_valid, X_test, Y_test

In [2]:
# loading the data from the csv files
# loading the data from the csv files
X_train, Y_train, X_valid, Y_valid, X_test, Y_test = load_data()
import numpy as np
import tensorflow
x_train = np.array(X_train, dtype='float32').reshape(-1, 11, 60, 100, 1)
x_valid = np.array(X_valid, dtype='float32').reshape(-1, 11, 60, 100, 1)
x_test = np.array(X_test, dtype='float32').reshape(-1, 11, 60, 100, 1)

onehot_encoder = OneHotEncoder()
    
Y_train_reshaped = np.array(Y_train).reshape(-1, 1)
Y_valid_reshaped = np.array(Y_valid).reshape(-1, 1)
Y_test_reshaped = np.array(Y_test).reshape(-1, 1)

Y_train_onehot = onehot_encoder.fit_transform(Y_train_reshaped)
Y_valid_onehot = onehot_encoder.fit_transform(Y_valid_reshaped)
Y_test_onehot = onehot_encoder.fit_transform(Y_test_reshaped)
    
# Convert one-hot encoded arrays to dense arrays
Y_train = Y_train_onehot.toarray()
Y_valid = Y_valid_onehot.toarray()
Y_test = Y_test_onehot.toarray()


nb_classes=Y_train


In [6]:
mean=x_train.mean(axis=0)
std=x_train.std(axis=0)

X_train=np.array((x_train-mean)/std)
X_valid=np.array((x_valid-mean)/std)
X_test=np.array((x_test-mean)/std)

print(X_train.shape)


(4768, 11, 60, 100, 1)


In [7]:
import tensorflow as tf


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv3D(64, kernel_size=(4, 5, 5), activation='relu', input_shape=(11, 60, 100, 1)))
model.add(tf.keras.layers.MaxPooling3D(pool_size=(2, 3, 4)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(nb_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
es=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='min', verbose=1, patience=5)

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(x_valid, y_valid,), callbacks=[es])

In [ ]:
from sklearn.metrics import classification_report

preds = model.predict(x_test)

print(classification_report(np.argmax(y_test,1),np.argmax(preds,1)))

In [ ]:
model.save('model.simple.keras')  

In [ ]:
model= tf.keras.models.load_model('model.simple.keras')

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization, Input, ReLU, GlobalAveragePooling3D, add
from keras.models import Model

output_shape = 6
input = Input(shape=(11, 60, 100, 1))

'''block_0'''
b0_conv3d_1 = Conv3D(64, kernel_size=(2, 3, 3), padding='same', use_bias=False,
                     name='b0_conv3d_1', kernel_initializer = 'he_normal')(input)
b0_relu_1 = ReLU(name='b0_relu_1')(b0_conv3d_1)
b0_bn_1 = BatchNormalization(name='b0_bn_1')(b0_relu_1)
b0_out =  MaxPooling3D(pool_size=(2, 2, 2))(b0_bn_1)

'''block_1'''
b1_cnv3d_1 = Conv3D(filters=16, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same',
                        use_bias=False, name='b1_cnv3d_1', kernel_initializer=HeNormal())(b0_out)
b1_relu_1 = ReLU(name='b1_relu_1')(b1_cnv3d_1)
b1_bn_1 = BatchNormalization(name='b1_bn_1')(b1_relu_1)  # size: 14*14

b1_cnv3d_2 = Conv3D(filters=32, kernel_size=(1, 1, 1), strides=(2, 2, 2), padding='same',
                    use_bias=False, name='b1_cnv3d_2', kernel_initializer=HeNormal())(b1_bn_1)
b1_relu_2 = ReLU(name='b1_relu_2')(b1_cnv3d_2)
b1_out = BatchNormalization(name='b1_out')(b1_relu_2)  # size: 14*14


'''block 2'''
b2_cnv3d_1 = Conv3D(filters=32, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same',
                    use_bias=False, name='b2_cnv3d_1', kernel_initializer=HeNormal())(b1_out)
b2_relu_1 = ReLU(name='b2_relu_1')(b2_cnv3d_1)
b2_bn_1 = BatchNormalization(name='b2_bn_1')(b2_relu_1)  # size: 14*14

b2_add = add([b1_out, b2_bn_1])  #

b2_cnv3d_2 = Conv3D(filters=64, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same',
                    use_bias=False, name='b2_cnv3d_2', kernel_initializer=HeNormal())(b2_add)
b2_relu_2 = ReLU(name='b2_relu_2')(b2_cnv3d_2)
b2_out = BatchNormalization(name='b2_bn_2')(b2_relu_2)  # size: 7*7

'''block 3'''
b3_cnv3d_1 = Conv3D(filters=64, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same',
                    use_bias=False, name='b3_cnv3d_1', kernel_initializer=HeNormal())(b2_out)
b3_relu_1 = ReLU(name='b3_relu_1')(b3_cnv3d_1)
b3_bn_1 = BatchNormalization(name='b3_bn_1')(b3_relu_1)  # size: 7*7

b3_add = add([b2_out, b3_bn_1])  #

b3_cnv3d_2 = Conv3D(filters=128, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same',
                    use_bias=False, name='b3_cnv3d_2', kernel_initializer=HeNormal())(b3_add)
b3_relu_2 = ReLU(name='b3_relu_2')(b3_cnv3d_2)
b3_out = BatchNormalization(name='b3_out')(b3_relu_2)  # size: 3*3


'''block 4'''
b4_avg_p = GlobalAveragePooling3D()(b3_out)
output = Dense(output_shape, name='model_output', activation='softmax',
                       kernel_initializer='he_uniform')(b4_avg_p)

model = Model(input, output)

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 11, 60,    │          0 │ -                 │
│ (InputLayer)        │ 100, 1)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b0_conv3d_1         │ (None, 11, 60,    │      1,152 │ input_layer_2[0]… │
│ (Conv3D)            │ 100, 64)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b0_relu_1 (ReLU)    │ (None, 11, 60,    │          0 │ b0_conv3d_1[0][0] │
│                     │ 100, 64)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b0_bn_1             │ (None, 11, 60,    │        256 │ b0_relu_1[0][0]   │
│ (BatchNormalizatio… │ 100, 64)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_2     │ (None, 5, 30, 50, │          0 │ b0_bn_1[0][0]     │
│ (MaxPooling3D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_cnv3d_1 (Conv3D) │ (None, 3, 15, 25, │     27,648 │ max_pooling3d_2[… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_relu_1 (ReLU)    │ (None, 3, 15, 25, │          0 │ b1_cnv3d_1[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_bn_1             │ (None, 3, 15, 25, │         64 │ b1_relu_1[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_cnv3d_2 (Conv3D) │ (None, 2, 8, 13,  │        512 │ b1_bn_1[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_relu_2 (ReLU)    │ (None, 2, 8, 13,  │          0 │ b1_cnv3d_2[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b1_out              │ (None, 2, 8, 13,  │        128 │ b1_relu_2[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_cnv3d_1 (Conv3D) │ (None, 2, 8, 13,  │      1,024 │ b1_out[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_relu_1 (ReLU)    │ (None, 2, 8, 13,  │          0 │ b2_cnv3d_1[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_bn_1             │ (None, 2, 8, 13,  │        128 │ b2_relu_1[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 2, 8, 13,  │          0 │ b1_out[0][0],     │
│                     │ 32)               │            │ b2_bn_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_cnv3d_2 (Conv3D) │ (None, 1, 4, 7,   │     55,296 │ add_4[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ b2_relu_2 (ReLU)    │ (None, 1, 4, 7,   │          0 │ b2_cnv3d_2[0][0]

 Total params: 313,286 (1.20 MB)

 Trainable params: 312,486 (1.19 MB)

 Non-trainable params: 800 (3.12 KB)

In [22]:
es=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', verbose=1, patience=10, restore_best_weights=True, min_delta=0.001)

In [24]:
model.fit(X_train, Y_train, batch_size=32, epochs=30, verbose=1, validation_data=(X_valid, Y_valid,), callbacks=[es])

Epoch 1/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 648s 4s/step - accuracy: 0.3641 - loss: 1.5984 - val_accuracy: 0.4523 - val_loss: 1.3474
Epoch 2/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 600s 4s/step - accuracy: 0.5891 - loss: 1.1200 - val_accuracy: 0.5394 - val_loss: 1.2146
Epoch 3/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 604s 4s/step - accuracy: 0.6349 - loss: 0.9655 - val_accuracy: 0.5519 - val_loss: 1.2040
Epoch 4/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 644s 4s/step - accuracy: 0.7180 - loss: 0.7779 - val_accuracy: 0.5394 - val_loss: 1.2159
Epoch 5/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 650s 4s/step - accuracy: 0.7467 - loss: 0.6801 - val_accuracy: 0.5145 - val_loss: 1.2363
Epoch 6/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 638s 4s/step - accuracy: 0.8029 - loss: 0.5536 - val_accuracy: 0.5643 - val_loss: 1.2788
Epoch 7/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 688s 5s/step - accuracy: 0.8565 - loss: 0.4254 - val_accuracy: 0.5519 - val_loss: 1.3351
Epoch 8/30
149/149 ━━━━━━━━━━━━━━━━━━━━ 595s 4s/step - accuracy: 0.8930 - loss: 0.3477 - val_accu

In [26]:
from sklearn.metrics import classification_report

preds = model.predict(X_test)

print(classification_report(np.argmax(Y_test,1),np.argmax(preds,1)))

8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 252ms/step
              precision    recall  f1-score   support

           0       0.65      0.56      0.60        50
           1       0.57      0.64      0.61        36
           2       0.52      0.46      0.49        50
           3       0.66      0.68      0.67        28
           4       0.70      0.70      0.70        50
           5       0.46      0.58      0.51        31

    accuracy                           0.60       245
   macro avg       0.59      0.60      0.60       245
weighted avg       0.60      0.60      0.60       245


In [28]:
model.save('model.complex.keras')

In [ ]:
model= tf.keras.models.load_model('model.complex.keras')